<h1 align="center">Advanced Image Processing and Analysis</h1>
<h3 align="center">ECE 4438B/ECE 9022/ECE 9202B/BIOMED 9519B/BIOPHYS 9519B/CAMI 9519B</h3><h4 align="center"><a href="mailto:echen29@uwo.ca"> Elvis Chen, PhD, LL</a></h4>
<h4 align="center">Day 11, February 11, 2019</h4>

### Introduction

At the end of Day 7 lecture (Region growing segmentation), we experimented with the concept of morphological operation to **clean up** the segmentation results.  Often, using histogram-based segmentation techniques, we are left with *holes* and other unwanted image artifacts in the segmented image. One possible reason for these artifacts to exist in segmentation is due to image noise: these holes correspond to pixel with intensity that falls outsize of the *normal* range (as compared to its surrounding neighbours). Thus techniques relying on thresholding or some statistical measures of the neighbouring pixel intensity won't be able to classify properly.

In this Jupyter Notebook, we will look into the basis for basic morphological operations, **examplified using binary images** including:

* [Dilate](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1BinaryDilateImageFilter.html)
* [Erode](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1BinaryErodeImageFilter.html)
* [Opening](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1BinaryMorphologicalOpeningImageFilter.html), and
* [Closing](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1BinaryMorphologicalClosingImageFilter.html) operations

These concepts can be applied to grayscale images as well:

* [Dilate](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1GrayscaleDilateImageFilter.html)
* [Erode](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1GrayscaleErodeImageFilter.html)
* [Opening](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1GrayscaleMorphologicalOpeningImageFilter.html), and
* [Closing](https://itk.org/SimpleITKDoxygen/html/classitk_1_1simple_1_1GrayscaleMorphologicalClosingImageFilter.html) operations

In [ ]:
import SimpleITK as sitk
import numpy as np

import matplotlib.pyplot as plt
%matplotlib notebook

# utilities for diplaying images
from myshow import myshow, myshow3d

**Morphological image processing** is a collection of *non-linear* operations related to the shape or morphology of features in an image. It is constructed with operations on sets of (neighbouring) pixels. **Binary** morphology uses only set membership and is indifferent to values, such as gray level or color, of a pixel. We will deal only with morphological operations for *binary* images for illustration purposes.

Let's look that what morphological image processing can do by ways of example:

In [ ]:
# Let us load an image of alphabets

# taken from http://documents.wolfram.com/applications/digitalimage/UsersGuide/Morphology/ImageProcessing6.3.html
img = sitk.ReadImage('wolf-glimage.png')
myshow(sitk.Expand(img, [2,2]))

In [ ]:
# Notice that even it is a grey-scale image, it is actually represented as a "color" image of 3 channels.
#
# Let us convert it into a single-channel image first.
img_Vector = sitk.GetArrayFromImage( img )
img_bw = sitk.GetImageFromArray(.299 * img_Vector[:,:,0] + .587 * img_Vector[:,:,1] + .114 * img_Vector[:,:,2])
img_bw.CopyInformation( img ) # why do we do this?
img_255 = sitk.Cast(sitk.RescaleIntensity(img_bw), sitk.sitkUInt8) 
myshow(sitk.Expand(img_255, [2,2]))

In [ ]:
# It is obviously bimodal, so Otsu's method should separate black from white easily
#
otsu_filter = sitk.OtsuThresholdImageFilter()
inside_value = 0
outside_value = 1
otsu_filter.SetInsideValue(inside_value)
otsu_filter.SetOutsideValue(outside_value)
seg = otsu_filter.Execute(img_255)
myshow(sitk.Expand(sitk.LabelOverlay(img_255, seg), [2,2]), title='Otsu Segmented Image') # the image may be too large to use myshow
print( otsu_filter.GetThreshold())

myshow(sitk.Expand(seg,[2,2]))

#### Erosion

Erosion is one of the two fundamental operations (the other being *dilation*) in morphological image processing from which all other morphological operations are based. It makes image features **thinner**.

In [ ]:
# A trivial example of Erosion
erodeFilter = sitk.BinaryErodeImageFilter()
erodeFilter.SetKernelType ( sitk.sitkCross )
erodeFilter.SetKernelRadius( [1,1]) # adjust the kernal size
img_eroded = erodeFilter.Execute( seg )
myshow( sitk.Expand(img_eroded, [2,2]))

#### Dilation

Dilation is the dual (i.e. opposite) of Erosion, it makes image features **thicker**.

In [ ]:
# a trivial example of dilation.
dilateFilter = sitk.BinaryDilateImageFilter()
dilateFilter.SetKernelType( sitk.sitkCross )
dilateFilter.SetKernelRadius( [1,1]) # adjust the kernel size
img_dilated = dilateFilter.Execute(seg)

myshow(sitk.Expand(img_dilated,[2,2]))

In [ ]:
# Let us load an binary image. The symble of Ying-Yang should be binary (0 for black, 255 for white)
img = sitk.ReadImage('..\\data\\images\\binary_scribble.png')

myshow(img)

In [ ]:
# It is obviously bimodal, so Otsu's method should separate black from white easily
#
otsu_filter = sitk.OtsuThresholdImageFilter()
inside_value = 1
outside_value = 0
otsu_filter.SetInsideValue(inside_value)
otsu_filter.SetOutsideValue(outside_value)
seg = otsu_filter.Execute(img)
myshow(sitk.LabelOverlay(img, seg), title='Otsu Segmented Image') # the image may be too large to use myshow
print( otsu_filter.GetThreshold())

In [ ]:
# create a binary (0 or 1) image
img_bw = img < otsu_filter.GetThreshold()
myshow(img_bw) # looks the same, doesn't it?

### Morphological operation

Morphological image processing is a collection of non-linear operations related to the *shape* or *morphology* of features in an image. The core idea is to *probe* an image with a simple, pre-defined shape, and then draw conclusion on how this shape fits or misses the shape in the image.  Similar to convolution of an image, these *probe* is called kernel (or **structuring element**), and itself is a binary image:

#### Structuring Element

A small binary image, i.e. a small matrix of pixels, each with a value of zero ($0$) or one ($1$)
* zero-values of the SE are ignored
* **Size** of the SE: the matrix dimensions
* **Shape** of the SE: pattern of ones and zeros
* **Origin** of the SE:: usually, one of its pixels, denoted in *red* in the picture below. The origin does *not* have to be in the center of the structuring element and, during the set operation, coincides with the pixel in the image for which the set operation is being performed.

A common practice is to have odd dimensions of structuring matrix and the origin defined as the centre of the matrix.

When an SE is placed in a binary image, each its pixel is associated with the pixel oof the area under the SE:
* The SE **fits** the image if **for each** of its pixel set to $1$ the corresponding image pixel is also $1$
* the SE **hits** (intersects) the image if **at least for one** of its pixels set to $1$ the corresponding image pixel is also $1$


<img src="SE_fit_hit.png" width="800"/> 

Zero-valued pixels of the structuring elements are ignored. Same applies when performing set operations on the border of the image.

#### Structuring Elements in SimpleITK

SimpleITK supports 4 Structuring elements:
* sitkAnnulus, a ring
* sitkBall, 
* sitkBox
* sitkCross

Each of which you can set the *kernel Radius*

A common practice is to have odd dimensions of structuring matrix and the origin defined as the centre of the matrix.

### Dilation

Let us look into morphological dilation first. The basic effect of a dilation operator on a binary image is to gradually enlarge the bondaries of regions of foreground pixels. Thus areas of foreground pixels grown in size while *holes within those regions becomes smaller*.

#### How it works

A more formal description of **Dilation** of a binary image $I$ by a structuring element $s$ (denoted as $I \oplus s$) produces a new binary image $J = I \oplus s$ with ones in all location $(x,y)$ of a structuring element's origin at which that structuring element $s$ **hits** the input image $I$, i.e. $J(x,y) = 1$ if $s$ hits $I$ and $0$ otherwise, repeating for all pixel coordinates $(x,y)$.  

Dilation adds a layer of pixels to both the inner and outer boundaries of regions.

In [ ]:
dilateFilter = sitk.BinaryDilateImageFilter()
dilateFilter.SetKernelType( sitk.sitkBall )
dilateFilter.SetKernelRadius( [5,5])
img_dilated = dilateFilter.Execute(img_bw)
myshow( img_dilated )

The holes enclosed by a single region and gaps between different regions become smaller, and small intrusions into boundaries of a regions are filled in:
- lines are now connected,
- holes in the big blob are filled.

### Erosion

Erosion is the *dual* of dilation. The basic effect of an erotion operator on a binary image is to gradually sharpen the boundaries of regions of foreground pixels. Thus areas of foreground pixels shrink in size.  As a result, isolated **islands** of foreground pixels will disappear.

#### How it works

A more formal description of **erosion** of a binary image $I$ by a structuring element $s$ (denoted as $I \ominus s$) produces a new binary image $J = I \ominus s$ with ones in all locations $(x,y)$ of a structuring element's origin at which that structuring element $s$ **fits** the input image $I$, i.e. $J(x,y)=1$ if $s$ fits $I$ and $0$ otherwise, repeating for all pixel coordinates $(x,y)$.

In [ ]:
erodeFilter = sitk.BinaryErodeImageFilter()
erodeFilter.SetKernelType ( sitk.sitkBall )
erodeFilter.SetKernelRadius( [1,1])
img_eroded = erodeFilter.Execute( img_bw )
myshow( img_eroded )

Erosion with small (e.g. $3\times 3$) square structuring elements shrinks an image by stripping away a layer of pixels from both the inner and outer boundaries of regions. The holes and gaps between different regions become more pronounced, and small details are eliminated:

- notices the small dots (noise?) are gone,
- holes in the large blog are larger.

Larger structuring elements have a more pronounced effect, the results of erosion with a large structuring element being *similar* to the result obtained by iterated erosion using a smaller structuring element of the same shape. If $s_1$ and $s_2$ are a pair of structuring element identical in shape, with $s_2$ twice the size of $s_1$, then:

$J \ominus s_2 \sim (I \ominus s_1) \ominus s_1$

While erosion removes small-scale details from a binary image, it also simultaneously reduces the size of regions of interest, too. By substrating the eroded image from the original image, boundaries of each region can be found:

$K = I - (I \ominus s)$

where $I$ is an image of the regions, $s$ the structuring element, and $K$ is an image of region boundaries.

In [ ]:
myshow( img_bw-img_eroded)

### Duality Between Dilation and Erosion

Dilation and erosion are *dual operations* in that they have opposite effects:
    
If a binary image is considered as a collection of connected regions of $1$ on backgrounds of $0$s:
* **Erosion** is the fitting of the SE to these regions
* **Dilation** the fitting of the SE (rotated if necessary) into the background, followed by inversion of the result.

Formally, the duality is written as:

$I\oplus s = I^{c} \ominus s_{rot}$

where $s_{rot}$ is the structuring elements $s$ rotated by $180^{o}$. If a structuring element is symmetrical with respect to rotation, then $s_rot$ does not differ from $s$. If a binary image is considered to be a collection of connected regions (foreground) of pixels set to $1$ on a background of pixels set to $0$, then erosion is fitting of a structuring element to these regions and dilation is the fitting of a structuring element (rotated if necessary) into the *background*, followed by inversion of the result.

### Compound Operations
Many morphological operations are represented as combinations of erosion, dilation, and simple set-theoretic operations (such as complement, intersections, and union. However, we will only look at morphological opening and closing operation.

### Opening

The **opening** of an image $I$ by a structuring element $s$ (denoted by $I \circ s$) is an erosion followed by a dilation:

$I \circ s = (I \ominus s) \oplus s$

It is useful in removing noise

In [ ]:
openingFilter = sitk.BinaryOpeningByReconstructionImageFilter()

openingFilter.SetKernelType ( sitk.sitkBall )
openingFilter.SetKernelRadius( [1,1])
img_opened = openingFilter.Execute( img_bw )
myshow( img_opened )

**Opening** is so called because it can open up gaps between objects connected by a thin bridge of pixels. Any regions that have survived the erosion are restored to their original size by the dilation

taken from [here](www.mmorph.com/html/morph/mmopen.html)
original binary image
<img src="img_mm_open4.gif" width="200"/> 

$I\circ s$ ($5\times 5$ square)
<img src="img_mm_open5.gif" width="200"/> 

$I\circ s$ ($9\times 9$ square)
<img src="img_mm_open6.gif" width="200"/> 

Morphological opening is an **idempotent** operation: once an image has been opened, subsequent openings with the same structuring element have no further effect on that image:

$(I \circ s) \circ s = I\circ s$

### Closing

The **closing** of an image $I$ by a structuring element $s$ (denoted by $I \bullet s$) is a **dilation** followed by an **erosion**:

$I \bullet s = ( I \oplus s) \ominus s$

It is useful to remove small (i.e. smaller than the structuring element) holes in the image.

In [ ]:
closingFilter = sitk.BinaryClosingByReconstructionImageFilter()

closingFilter.SetKernelType ( sitk.sitkBall )
closingFilter.SetKernelRadius( [1,1])
img_closed = closingFilter.Execute( img_bw )
myshow( img_closed )

Closing is so called because it can fill holes in the regions while keeping the initial region size. Like opening, closing is **idempotent**:

$(I \bullet s) \bullet s = I \bullet s$

and it is dual operation of opening (just as opening is the dual operation of closing):

$I\bullet s = (I^{c} \circ s)^{c}, I \circ s = (f^{c} \bullet s)^{c}$

In other words, closing (opening) of a binary image can be performed by taking the complement of that image, opening (closing) with the structuring element, and taking the complement of the result.